In [ ]:
#import cupy as cp
import qutip as qt
import numpy as np
import sympy as sp
import time
import math
import scipy
from qutip.qobj import Qobj
import matplotlib.pyplot as plt

from qiskit.tools.visualization import plot_histogram
from qiskit.extensions import Initialize
from qiskit.tools.monitor import job_monitor
from qiskit.quantum_info import DensityMatrix, Statevector
from qiskit.circuit.quantumcircuit import QuantumCircuit
from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister, ClassicalRegister, transpile
import itertools
from qiskit.quantum_info import DensityMatrix as dm
from qiskit.quantum_info import state_fidelity as fid
from qiskit_aer.noise import NoiseModel
import itertools as it
from sparse import COO
from tqdm import tqdm
import pandas as pd

In [ ]:
from qiskit_ibm_provider import IBMProvider
from qiskit import IBMQ
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#IBMProvider.save_account('',overwrite=True)

In [ ]:
#provider = IBMProvider()

In [ ]:
#qcomp = provider.get_backend('ibmq_lima')

# Helper Functions

In [ ]:
# Standard Tomography

def pauli_basis(n):
    import functools as ft
    I= np.array([[1,0],[0,1]])
    Z= np.array([[1,0],[0,-1]])
    X= np.array([[0,1],[1,0]])
    Y= np.array([[0,-1j],[1j,0]])
    meas = list(it.product(['I','X','Y','Z'], repeat=n))
    Am = [list(i) for i in meas]
    basis=[]
    for i in Am:
        a = ft.reduce(np.kron,list(map(eval, i)))
        basis.append(a)
    return np.array(basis), Am


def reconstruct_rho(counts,basis):
   return np.tensordot(counts,basis,axes=1)

def get_U(basis):
    ei,ve=np.linalg.eig(basis)
    d = basis.shape[0]
    U = np.zeros((d,d),dtype=complex)
    for j in range(d):
        B=ve[:,j]
        B_h=np.array([B])              
        U[j,:]=ei[j]*B_h
    return U, ei

#def validate_rho(rho_est):
  
#    s,v,d = np.linalg.svd(rho_est)
#   v = v*0
#    v[0] = 1

#    return np.matmul(np.matmul(s,np.diag(v)),d)

def validate_rho(rho_est):
    ei,ve = np.linalg.eig(rho_est)
    idx = np.argmax(ei)
    v = ve[:,idx]
    return np.outer(v,v.conj())

def std_qst(n,PSI,N):
    BASIS,Am = pauli_basis(n)
    counts = []

    for i in range(len(BASIS)):
        U, ei = get_U(BASIS[i])
        f_ibm = IBM_counts(n,PSI,U,N,con=True)
        counts.append(np.real(np.dot(f_ibm,ei)))
    return validate_rho(reconstruct_rho(np.array(counts),np.array(BASIS)).conj()/d)

def mc_qst(n,PSI,N):
    d = 2**n
    rho_est=np.zeros((d,d),dtype = 'complex_')

    rho_est=IBM_M(rho_est,N,False)  # Real Basis
    rho_est=IBM_M(rho_est,N,True) # Complex Basis
    
    f_ibm_z = IBM_counts(n,PSI,np.eye(d),N,con=False) # IBM Z-basis     
    
    mc_st = time.time()
    rho_est = rho_est.reshape(d,d)    
    for i in range(d):
        rho_est[i,i] = np.abs(f_ibm_z[i]) 
    rho_Est = Mat_C(rho_est) # Matrix Completion
    mc_et = time.time()
    return validate_rho(rho_Est),   mc_et-mc_st
    #return rho_Est

def mc_qst_v2(n,PSI,N,params):
    d = 2**n
    rho_est=np.zeros((d,d),dtype = 'complex_')

    rho_est=IBM_M(rho_est,N,False)  # Real Basis
    rho_est=IBM_M(rho_est,N,True) # Complex Basis
    
    f_ibm_z = IBM_counts(n,PSI,np.eye(d),N,con=False) # IBM Z-basis     
    mc_st = time.time()
    rho_est = rho_est.reshape(d,d)    
    for i in range(d):
        rho_est[i,i] = np.abs(f_ibm_z[i]) 
    rho_Est = rank_approx(rho_est,params) # Matrix Completion
    mc_et = time.time()
    return validate_rho(rho_Est),  mc_et-mc_st
    #return rho_Est

def get_X(n):
    d = 2**n
    BASIS,Am = pauli_basis(n)
    BASIS = np.delete(BASIS,0,0)
    Am.pop(0)
    ent = np.tile([1,-1],d**3-d)
    temp = np.zeros((d**2-1,d**3-d))
    ct = 0
    k = 0

    
    for i in range(len(ent)):  
        if i%d == 0 and i != 0:
            ct += 1
            if ct == d**2-1:
                break
        if (np.any(np.array(Am[ct]) == 'X') or np.any(np.array(Am[ct]) == 'Y')):
            temp[ct,i] = ent[i]
        else:
            e = np.linalg.eigvals(BASIS[ct])
            if k%d==0:
                k=0
                temp[ct,i] = e[k]
                k+=1
            else:
                temp[ct,i] = e[k]
                k+=1
            
        
             
            
    return temp, BASIS, Am
    
    


def LRE_qst(n,PSI,N):
    X, BASIS,Am = get_X(n)
    counts = []
    d = 2**n
    for i in range(len(BASIS)):
        U, ei = get_U(BASIS[i])
        f_ibm = IBM_counts(n,PSI,U,N,con=True)
        #pp = LRE_IBM(n,PSI,Am[i],N)
        #print(f_ibm)
        #print(pp)
        counts.append(f_ibm-(1/d))
    st_std = time.time()
    Y = np.real((np.linalg.inv((X@X.T)) @ X) @ np.array(counts).reshape(d**3-d,1)).flatten()
    rho_lre = validate_rho(reconstruct_rho(Y,BASIS).conj()+ np.eye(d,d)/d)
    end_std = time.time()
    return      rho_lre, end_std-st_std
    
def Mat_C(rho_est):

    ind=3
    for x in range(n-1):
        for y in range(1,2**(x+1)):
            #x = d
       
            if rho_est[0,ind-2**(x+1)] ==  rho_est[y,ind-2**(x+1)]:
                rho_est[0,ind] = rho_est[y,ind]
            elif rho_est[0,ind-2**(x+1)] == 0:
                rho_est[0,ind] = 0
            else:
                rho_est[0,ind]=rho_est[y,ind]*rho_est[0,ind-2**(x+1)]/(rho_est[y,ind-2**(x+1)]+10**(-20))
    
            rho_est[ind,0]=rho_est[0,ind].conj()
            ind=ind+1
        ind=ind+1;   
    iter=0
    A= rho_est[:,iter]
    B= rho_est[iter,:]
    D= rho_est[iter,iter]
    rho_Est= np.outer(A,B)/ D
    
    return rho_Est


def rank_approx(rho_est,params):
    k = params['k']
    r = params['r']
    tol = params['tol']
    patience = params['patience']
    #fill all teh zero entries with the mean of it's column
    Z = np.array(rho_est)
    #I is an indicator matrix: 1 for missing; 0 otherwise
    I = Z==0
    for i in range(d):
        Z[i,I[i]] = np.mean(Z[i,~I[i]])

    error = 1
    for i in range(k):
        U, s, V = np.linalg.svd(Z, full_matrices=True)
        s_dash = s[r]
        s_new = s - s_dash
        s_new[s_new<0] = 0
        if i>0:
            error = np.mean(np.abs(s_new-s_temp)**2)
        
        s_temp = np.array(s_new)
        
        W = np.dot(U, np.dot(np.diag(s_new), V))
        Z[I] = W[I]
        #break if the s_new does'nt change much
        if  error < tol:
            patience-=1
            if patience == 0:
                print('Converged at iteration: ',i)
                break
            
    return Z       

def h_trans(n):
    H = (1/np.sqrt(2))*np.array([[1,1],[1,-1]])
    I = np.eye(2)
    if n==1:
        A = H
    else:
        A = np.kron(H,I)
        for i in range(n-2):
            A = np.kron(A,H)
    return A    

def get_keys(n,t):
    keys = []
    temp =[]
    c = np.zeros(2**n, dtype=int)
    for i in range(2**n):
        keys.append(format(i, '0'+str(n)+'b'))
        temp = keys
        for j in range(t):
            temp[i] = temp[i][1:] + temp[i][0]
        c[i] = int(temp[i], 2)
    return keys,c





def IBM_M(rho_est,shots,con):
    for i in range(n):
        f_IBM = IBM_MCQST(n,PSI,i,shots,con)
        keys,indx = get_keys(n,n-1-i)
  
        
        probs = np.array(list( map(f_IBM.get, keys,[0]*len(keys)) ))/shots
        ct = 0
   
        for i in range(d//2):
            val = (probs[ct] - probs[ct+1])
            if con:
                fct = 1j*0.5
            else:
                fct = 0.5
            rho_est[indx[ct],indx[ct+1]] = rho_est[indx[ct],indx[ct+1]]+fct* (val)
            rho_est[indx[ct+1],indx[ct]] = rho_est[indx[ct],indx[ct+1]].conj()
            ct+=2
    #display(sp.Matrix(rho_est.reshape(d,d)))
    return rho_est

# Experiment Setup 

In [ ]:
def print_report(prot,rho,rho_est,Time):
    print("Time taken by ",str(prot), np.round(Time,3), 'secs')
    print('Infidility : ', 1-fid(rho,rho_est))
    print('########################################') 
    print('                                        ')


def IBM_counts(n,rho,meas,sh,con):
   # rho_dm = dm(rho)
    idx = list(np.arange(n))
    
    
    sim_noise = qcomp
    counts=[]
    circuit = QuantumCircuit(n)
    circuit.initialize(rho,idx)
    if con:
        circuit.unitary(meas,idx)
    circuit.measure_all() 
    
    job = execute(transpile(circuit, sim_noise),
                 sim_noise, shots=sh)
    counts=job.result().get_counts()
    keys,_ = get_keys(n,n)    
    probs = np.array(list( map(counts.get, keys,[0]*len(keys)) ))/sh
    
    return probs

def IBM_MCQST(n,PSI,loc,N,con):
    qc = QuantumCircuit(n)
    qc.initialize(PSI,range(n))
    if con == True:
        qc.s(-loc-1)
    qc.h(-loc-1)
    qc.measure_all()
    job = execute(transpile(qc, qcomp),
                 qcomp, shots=N)
    counts=job.result().get_counts()
    return counts

#   Quantum Computer Configuration

In [ ]:
#qcomp = BasicAer.get_backend('qasm_simulator')
#qcomp = provider.get_backend('ibm_lagos')
#backend = provider.get_backend('ibm_lagos')
#Real Computer Noise Model
#noise_model = NoiseModel.from_backend(backend)
# Get coupling map from backend
#coupling_map = backend.configuration().coupling_map
# Get basis gates from noise model
#basis_gates = noise_model.basis_gates
simulator_gpu = Aer.get_backend('aer_simulator')
#simulator_gpu.set_options(device='GPU')
qcomp = simulator_gpu

# Experiments

In [ ]:
avg=10 # number of averages
n=4 # number of qubits    
d=2**n # dimension of the Hilbert space

Fid     =    np.zeros((avg,2)) # Fidelity
Time    =    np.zeros((avg,2)) # Time
avg_fid = []
avg_time = []
cop = []
po = 1
for c in range(po):
    N = 1000000 * (10**c) # number of copies
    for k in tqdm(range(avg)):
     ########### Config ###########
        
        
        psi=np.array(qt.rand_ket_haar(d)) # random state
        #psi = np.load(str(n)+'_psi.npy')
        #psi = qt.ghz_state(n).full()
        #psi = np.array([-0.00827484+0.54549734*1j,  0.0030454 -0.12905376*1j, 0.64517208-0.27475179*1j,  0.14379161-0.41627929*1j])
        rho=np.outer(psi,psi.conj())
        
        PSI=Statevector(psi)
        trans = False
        if  np.any(np.diag(rho),0):
            PSI = PSI.evolve(h_trans(n))
            trans = True
        ##############################
        
        ########### LRE ###########
        
        shots_lre = np.floor(N/(d**2-1))
        rho_est_sd,time_std=LRE_qst(n,Statevector(psi),shots_lre)
        ##############################
        
        
        ########### Matrix Completion ###########
        shots_mc = np.floor(N/(2*n+1))
        itera = 10000             #number of iterations
        r = 1               #rank of the matrix
        tol = 1e-10          #tolerance
        patience = 10       #patience for convergence
        params = {'k':itera,'r':r,'tol':tol,'patience':patience}
        #rho_est_mc = mc_qst(n,PSI,shots_mc)
        rho_est_mc,time_mc = mc_qst_v2(n,PSI,shots_mc,params)
        ##############################
        if trans:
            rho_est_mc = dm(rho_est_mc).evolve(h_trans(n).conj().T).data
            
            
        Fid[k,0] = 1-fid(rho,rho_est_mc)
        Time[k,0] = time_mc
        
        Fid[k,1] = 1-fid(rho,rho_est_sd)
        Time[k,1] = time_std
        print('########################################')
        print('MC  : ', Fid[k,0], 'Time: ', Time[k,0])
        print('LRE : ', Fid[k,1], 'LRE: ', Time[k,1])
        print('########################################')
        

        
        
    avg_fid.append(np.median(Fid,axis=0))
    avg_time.append(np.median(Time,axis=0))
    cop.append(N)
    print('MC  : ', np.median(Fid,axis=0)[0], 'Time: ', np.median(Time,axis=0)[0])
    print('LRE : ', np.median(Fid,axis=0)[1], 'LRE: ', np.median(Time,axis=0)[1])
    print('Copies : ', N)
    print('########################################')
    #print_report('LRE',rho,rho_est_sd,end_std-st_std)
    #print_report('MC',rho,rho_est_mc,end_mc-st_mc)

#fid_dict = {'Copies':cop, 'MC':np.array(avg_fid)[:,0] , 'LRE':np.array(avg_fid)[:,1]}
#time_dict = {'Copies':cop, 'MC':np.array(avg_time)[:,0] , 'LRE':np.array(avg_time)[:,1]}

##df_fid = pd.DataFrame(fid_dict)
#df_time = pd.DataFrame(time_dict)

#df_fid.to_csv(str(n)+'_fid.csv', index=False)
#df_time.to_csv(str(n)+'_time.csv', index=False)

In [ ]:
sp.Matrix(rho_est_mc)